In [1]:
from yolo import YOLO
from utils import get_annotations_images, LabelEncoder, read_labels
from config import Config
from networkfactory import NetworkFactory
from batchgenerators import batch_generator, batch_generator_inmemory
from filter_dataset import filter_datasaet

Using TensorFlow backend.


In [2]:
annotation_folder = r'.\annotations'
images_folder = r'.\images'

annotations, images = get_annotations_images(annotation_folder, images_folder)
print(len(annotations))

5011


In [3]:
images_dir = r'./images'
annotations_dir = r'./annotations'
image_sets_dir = r'./imagesets'

wanted_sets = ['bicycle', 'bus', 'car', 'motorbike', 'person']

annotations_sdc, images_sdc = filter_datasaet(images_dir, annotations_dir, image_sets_dir, wanted_sets)
print(len(annotations_sdc))

2724


In [4]:
labels_dir = "./labels.txt"
cfg_path = r"./cfg/mobilenetyolov2.cfg"
weights = r"./weights/mobilenetyolov2_take2"
#weights = r"./weights/mobilenetyolov2-checkpoint-50"


cfg = Config(cfg_path)
encoder = LabelEncoder(read_labels(labels_dir)[0])
networkfactory = NetworkFactory()

#yolo = YOLO(cfg, encoder, networkfactory)
yolo = YOLO(cfg, encoder, networkfactory, weights)

C:\Users\Gencho\Anaconda3\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0321 20:48:28.986797 10476 deprecation.py:506] From C:\Users\Gencho\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0321 20:49:02.360071 10476 deprecation.py:323] From C:\Users\Gencho\Desktop\ObjectDetection\yolo.py:111: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0321 20:49:02.426069 10476 deprecation.py:32

In [ ]:
epochs = 250

#train(yolo, batch_generator_inmemory, annotations[:300], images[:300] , epochs, checkpoint_period=5, early_stopping=True)
yolo.train(batch_generator, annotations, images, epochs, checkpoint_period=5, early_stopping=True)

In [ ]:
yolo.save(weights)